# Signal e- Run 05 Analysis

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser
import os
import re
import time
from sys import getsizeof

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import copy

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools
init_notebook_mode(True)

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo

from mu2e import mu2e_ext_path
from mu2e.dataframeprod import DataFrameMaker
from mu2e.mu2eplots import mu2e_plot, mu2e_plot3d

In [ ]:
data_dir = "/home/ckampa/Coding/Mu2E_Extras/trajectory/run05/"
plot_dir = "/home/ckampa/Coding/Mu2E_Extras/plots/trajectory/run05/"
run_name = "trajectory_run05_"

## Prepping Trajectory Run 04
- And initial plotting.

### Plot All Trajectories

**NOTE: This takes a few minutes to run so don't rerun if you don't need to**

In [ ]:
subruns = sorted([i for i in os.listdir(data_dir+'raw') if "subrun" in i])

In [ ]:
subruns

In [ ]:
df_out = pd.read_pickle(data_dir+f"sparse/subrun02/trj.sparse.run_05.subrun_02.event_128.p")

In [ ]:
df_init = pkl.load(open(mu2e_ext_path+'trajectory/run05/trajectory_run05_INIT_COND.p','rb'))

In [ ]:
df_init.describe()

In [ ]:
figs = pkl.load(open(plot_dir+"tracks/plotly_run05_subrun01.p","rb"))

In [ ]:
plot(figs[5])

In [ ]:
#plot(figs[10])

In [ ]:
# stopping target
stop_zs = np.linspace(5500.,6300.,34)
r = 75.
r_in = 25.
phis = np.linspace(0,2*np.pi,50)
xx = r*np.cos(phis)
yy = r*np.sin(phis)
x_in = r_in*np.cos(phis)
y_in = r_in*np.sin(phis)

# tracker
trk_zs = np.linspace(8410.,11660.,18)
r_tr = 700.
r_tr_in = 400.
x_tr = r_tr*np.cos(phis)
x_tr_in = r_tr_in*np.cos(phis)
y_tr = r_tr*np.sin(phis)
y_tr_in = r_tr_in*np.sin(phis)

# run num
run_num = run_name[-3:-1]

In [ ]:
## SPARSIFY!
# Runs quite quickly
for subrun in subruns:
    fig_list = []
    for particle in sorted(os.listdir(data_dir+"raw/"+subrun)):
        #print(f"Run: 05, Subrun: {subrun}, Event: {particle}")
        df_out = pd.read_pickle(data_dir+"raw/"+subrun+f"/{particle}")
        df_out = pd.concat([df_out[::10],df_out.tail(1)],axis=0)
        df_out.reset_index(drop=True,inplace=True)
        df_out.to_pickle(data_dir+"sparse/"+subrun+f"/{particle.replace('raw','sparse')}")

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
num_cores = multiprocessing.cpu_count()

In [ ]:
## PLOTS!
## Takes a long time, so running in chunks of subruns.
#for subrun in subruns:#[2:]#[0:2]:
def gen_plots(subrun):
    fig_list = []
    plots_exist = os.listdir(plot_dir+f"tracks/{subrun}")
    #print(plots_exist)
    for particle in sorted(os.listdir(data_dir+"raw/"+subrun)):
        filename = f"plot.track.{particle.replace('trj.raw.','')}"
        if filename+".pdf" in plots_exist:
            continue
        df_out = pd.read_pickle(data_dir+"sparse/"+subrun+f"/{particle.replace('raw','sparse')}")
        #print(df_out.head())
        #df_out = pd.concat([df_out[::10],df_out.tail(1)],axis=0)
        #df_out.reset_index(drop=True,inplace=True)
        #df_out.to_pickle(data_dir+"sparse/"+subrun+f"/{particle.replace('raw','sparse')}")
                         
                         
        # plotting
        if df_out.Z.min() > 5500.:
            zm = 5500.
        else:
            zm = df_out.Z.min()
                         
        fig = scatter3d([df.X for df in [df_out[::2]]],
          [df.Y for df in [df_out[::2]]],
          [df.Z for df in [df_out[::2]]],
          scale_list=[df.time for df in [df_out[::2]]],
          mode_list=['markers+lines'],units_list=[('mm','mm','mm')],inline=True, 
          colors_list = ["Viridis"], aspect_auto=False, show_plot=False,
          rangex=(700+700),rangey=(700+700),rangez=(12000-zm),opacity_list=[1.]
         );
        camera = dict(
                up=dict(x=0, y=1, z=0),
                center=dict(x=0, y=0, z=-0.3),
                eye=dict(x=-3.6, y=1., z=-0.5)#(x=-5,y=1.5,z=-0.25)#(x=-1.8, y=0.2, z=-0.1)
        )
        for z_s in stop_zs:
            zst = np.ones_like(phis)*z_s
            fig.add_mesh3d(
                    x=xx,
                    y=yy,
                    z=zst,
                    opacity=0.3,
                    color='red',
                )
            fig.add_mesh3d(
                    x=x_in,
                    y=y_in,
                    z=zst,
                    opacity=0.3,
                    color='white',
                )
        for z_t in trk_zs:
            ztr = np.ones_like(phis)*z_t
            fig.add_mesh3d(
                    x=x_tr,
                    y=y_tr,
                    z=ztr,
                    opacity=0.15,
                    color='grey',
                )
            fig.add_mesh3d(
                    x=x_tr_in,
                    y=y_tr_in,
                    z=ztr,
                    opacity=0.15,
                    color='white',
                )

        fig.layout.showlegend = False
        fig.layout.dragmode = 'orbit'
        fig.layout.scene.camera = camera
        fig.layout.title = f"Run: {run_num}, Subrun: {subrun[-2:]}, Event: {particle[-5:-2]}"
                                         
        pio.write_image(fig,plot_dir+"tracks/"+subrun+"/"+filename+'.pdf')
        fig_list.append(fig)
        #plot(fig,filename=plot_dir+"tracks/"+subrun+"/"+filename+'.html',auto_open=False)
    pkl.dump(fig_list,open(plot_dir+f"tracks/plotly_run05_{subrun}.p","wb"))
    print(f"{subrun} completed.")

In [ ]:
start = time.time()
Parallel(n_jobs=num_cores)(delayed(gen_plots)(subrun) for subrun in subruns)
stop = time.time()

In [ ]:
(stop-start)/60

In [ ]:
(stop-start)/60

In [ ]:
figs = pkl.load(open(plot_dir+"tracks/plotly_run05_subrun10.p","rb"))

In [ ]:
fig = figs[2]

### Individual Interactive Plots
- Use general method from above to generate on the fly!
- Quick test of the new "Plotly Express"
    - Careful on this though because it seems they may be pushing to separate into different package. Should be pretty easy to fix later if necessary.

#### Plotly Express

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(df_out, x="X",y="Y",z="Z",color="time")

In [ ]:
fig.show()

In [ ]:
px.histogram(df_out, x="Z",nbins=50)

In [ ]:
px.colors.sequential.swatches()

Seems like a very nice API that I may have to start using!

#### Single Plots

In [ ]:
plot_dir = f"{mu2e_ext_path}plots/trajectory/run05/"
# stopping target
stop_zs = np.linspace(5500.,6300.,34)
r = 75.
r_in = 25.
phis = np.linspace(0,2*np.pi,50)
xx = r*np.cos(phis)
yy = r*np.sin(phis)
x_in = r_in*np.cos(phis)
y_in = r_in*np.sin(phis)

# tracker
trk_zs = np.linspace(8410.,11660.,18)
r_tr = 700.
r_tr_in = 400.
x_tr = r_tr*np.cos(phis)
x_tr_in = r_tr_in*np.cos(phis)
y_tr = r_tr*np.sin(phis)
y_tr_in = r_tr_in*np.sin(phis)

In [ ]:
fig_geom = go.Figure()
for z_s in stop_zs:
    zst = np.ones_like(phis)*z_s
    fig_geom.add_mesh3d(
            x=xx,
            y=yy,
            z=zst,
            opacity=0.3,
            color='red',
        )
    fig_geom.add_mesh3d(
            x=x_in,
            y=y_in,
            z=zst,
            opacity=0.3,
            color='white',
        )
for z_t in trk_zs:
    ztr = np.ones_like(phis)*z_t
    fig_geom.add_mesh3d(
            x=x_tr,
            y=y_tr,
            z=ztr,
            opacity=0.15,
            color='grey',
        )
    fig_geom.add_mesh3d(
            x=x_tr_in,
            y=y_tr_in,
            z=ztr,
            opacity=0.15,
            color='white',
        )

In [ ]:
subruns = sorted([i for i in os.listdir(data_dir+'raw') if "subrun" in i])

In [ ]:
@interact
def gen_plot(#run=["run05"],
             subrun=subruns,
             event=[f"{i:03d}" for i in range(200)]):#, 
             #new_window = False):
    run = "run05"
    data_dir = mu2e_ext_path+f"trajectory/{run}/sparse/{subrun}/"
    files = os.listdir(data_dir)
    file = [f for f in files if f"{event}" in f][0]
    
    df_out = pd.read_pickle(data_dir+file)
    df_out = pd.concat([df_out[::2],df_out.tail(1)],axis=0)
    if df_out.Z.min() > 5500.:
        zm = 5500.
    else:
        zm = df_out.Z.min()                 
    fig = scatter3d([df.X for df in [df_out]],
      [df.Y for df in [df_out]],
      [df.Z for df in [df_out]],
      scale_list=[df.time for df in [df_out]],
      mode_list=['markers+lines'],units_list=[('mm','mm','mm')],inline=True, 
      colors_list = ["Viridis"], aspect_auto=False, show_plot=False,
      rangex=(700+700),rangey=(700+700),rangez=(12000-zm),opacity_list=[1.],
      fig_ = fig_geom, copy_fig = True,
     );
    camera = dict(
            up=dict(x=0, y=1, z=0),
            center=dict(x=0, y=0, z=-0.3),
            eye=dict(x=-3.6, y=1., z=-0.5)#(x=-5,y=1.5,z=-0.25)#(x=-1.8, y=0.2, z=-0.1)
    )
   
    fig.layout.showlegend = False
    fig.layout.dragmode = 'orbit'
    fig.layout.scene.camera = camera
    fig.layout.title = f"Run: {run[-2:]}, Subrun: {subrun[-2:]}, Event: {event}"
    
    #if new_window:
    #    plot(fig,filename=plot_dir+"tracks/track_interactive_temp.html")
    return fig

In [ ]:
gen_plot("run05","subrun04","104")

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f gen_plot gen_plot("run05","subrun10","063")

In [ ]:
%prun gen_plot("run05","subrun10","063")

### Concat Files to one DataFrame

In [ ]:
subruns = sorted([i for i in os.listdir(data_dir+'raw') if "subrun" in i])

In [ ]:
r_init = re.compile("trajectory.*INIT_COND\.p$")
#r_data = re.compile("trajectory.*\d\d\d\.p$")
r_data = []
for subrun in subruns:
    for particle in sorted(os.listdir(data_dir+"sparse/"+subrun)):
        r_data.append(f"{data_dir}sparse/{subrun}/{particle}")   
#r_data = [i+"_Bshift-25mm-NONE.p" for i in particle_ids]
#directory04 = mu2e_ext_path+'trajectory/run04/'

In [ ]:
len(r_data)

In [ ]:
init_05 = [data_dir+f for f in os.listdir(data_dir) if r_init.match(f)]
data_05 = r_data
#data_04 = sorted([data_dir+i for i in r_data])#sorted([data_dir+f for f in os.listdir(data_dir) if r_data.match(f)])

In [ ]:
init_05

In [ ]:
data_05

Now we have lists of files to read in and have separated them by run and type of file. Now we can load in everything and concatenate!

In [ ]:
# run 03
# load in all dataframes
srun = re.compile("subrun\d\d")
evt = re.compile("\d\d\d")
#srun.findall(f)[0]

df = []
for fname in data_05:
    df_ = pkl.load(open(fname,'rb'))
    df_.eval('run = 5', inplace=True)
    df_.eval(f'subrun = {int(srun.findall(fname)[0][-2:])}',inplace=True)
    df_.eval(f'event = {int(evt.findall(fname)[0])}',inplace=True)
    df_.run = df_.run.astype('category')
    df_.subrun = df_.subrun.astype('category')
    df.append(df_)

In [ ]:
# concat!
df = pd.concat(df, join='outer', ignore_index=True, sort = False)

In [ ]:
df.eval('R = (X**2+Y**2)**(1/2)',inplace=True)

In [ ]:
df

In [ ]:
# init conditions
df_init = []
# run03
df_ = pkl.load(open(init_05[0],'rb'))
df_.eval('run = 5', inplace=True)
df_['subrun'] = pd.Series([i // 200 + 1 for i in range(len(df_))])
df_['event'] = pd.Series([i % 200 for i in range(len(df_))])#df_.index.astype('category')
df_.run = df_.run.astype('category')
df_.subrun = df_.subrun.astype('category')
df_init.append(df_)

In [ ]:
df_init = pd.concat(df_init, join='outer', ignore_index=True, sort = False)

In [ ]:
df_init.eval('particle = index',inplace=True)
df_init.particle = df_init.particle.astype('category')

nev = len(df_init.event.unique())

df.eval(f'particle = (subrun-1)*{nev} + event',inplace=True)
df.particle = df.particle.astype('category')

In [ ]:
df.to_pickle(data_dir+"trj.sparse.all.base.p")

In [ ]:
df_init.to_pickle(data_dir+"trj.init.all.base.p")

## Main Analysis

In [ ]:
# read in prepped df
df = pd.read_pickle(data_dir+"trj.sparse.all.base.p")
df_init = pd.read_pickle(data_dir+"trj.init.all.base.p")

In [ ]:
# check size in MB
getsizeof(df) / 1000 / 1000

In [ ]:
getsizeof(df_init) / 1000 / 1000

In [ ]:
df_init

In [ ]:
df

In [ ]:
#df_init[df_init.event == 0]
df_init[df_init.particle == 0]

Now we can get some general ideas in trends of positional data. An easy case is how conversion e- populate X and Y. The full data is overkill and a bit unwieldy, so we'll select every 10 data points (1e-10 s).
**Already done on import of sparse sets.**

In [ ]:
#df_small = df.iloc[::10,:]

In [ ]:
data = [
    #go.Histogram2dContour(
    go.Histogram2d(
        x=df['X'],
        y=df['Y'],
        colorscale='Viridis',
        colorbar=dict(
            title="Count"
        )
        #bins
        #line=line(width=0.1)
    )
]

axis_template = dict(
    ticks='',
    showgrid=False,
    zeroline=False,
    showline=True,
    mirror=True,
    linewidth=2,
    linecolor='#444'
)

xax = copy.copy(axis_template)
xax["title"] = "X (mm)"
yax = copy.copy(axis_template)
yax["title"] = "Y (mm)"

layout = go.Layout(xaxis=xax,#axis_template.,
                   yaxis=yax,#axis_template,
                   width=800,
                   height=750,
                   autosize=False,
                   hovermode='closest',
                   title='Signal e- Trajectory 2d Histo, 1e-10 s Time Steps'
)

fig = go.Figure(data=data, layout=layout)

In [ ]:
iplot(fig, filename=mu2e_ext_path+"trajectory/xy_2d_histo_sparse2.html")

In [ ]:
fname = f"{plot_dir}run05_xy_2d_histo_1e-10s_steps"
#pio.write_html(fig, fname+".html")
pio.write_image(fig, fname+".pdf")
pio.write_image(fig, fname+".png")

### Removing Low Radius, Small Z Events

In [ ]:
df

In [ ]:
grouped = df.groupby("particle")

In [ ]:
#grouped.describe()

In [ ]:
#grouped.R.max()[grouped.R.max()>400.]

In [ ]:
zcut = grouped.R.max()[grouped.Z.max()>8410.]

In [ ]:
#zcut

In [ ]:
zcutrcut = zcut[zcut > 400.]

In [ ]:
largeR_tracks = list(zcutrcut.index.values)

In [ ]:
len(largeR_tracks)

In [ ]:
largeR_cut = df.particle.isin(largeR_tracks)

In [ ]:
data = [
    #go.Histogram2dContour(
    go.Histogram2d(
        x=df[largeR_cut]['X'],
        y=df[largeR_cut]['Y'],
        colorscale='Viridis',
        colorbar=dict(
            title="Count"
        )
        #bins
        #line=line(width=0.1)
    )
]

axis_template = dict(
    ticks='',
    showgrid=False,
    zeroline=False,
    showline=True,
    mirror=True,
    linewidth=2,
    linecolor='#444'
)

xax = copy.copy(axis_template)
xax["title"] = "X (mm)"
yax = copy.copy(axis_template)
yax["title"] = "Y (mm)"

layout = go.Layout(xaxis=xax,#axis_template.,
                   yaxis=yax,#axis_template,
                   width=800,
                   height=750,
                   autosize=False,
                   hovermode='closest',
                   title='Signal e- Trajectory 2d Histo, 1e-10 s Time Steps <br> R_max > 400 mm, Z_max > 8410 mm'
)

fig = go.Figure(data=data, layout=layout)

In [ ]:
iplot(fig, filename=mu2e_ext_path+"trajectory/largeR_largeZ_xy_2d_histo_sparse.html")

In [ ]:
fname = f"{plot_dir}run05_largeR_largeZ_xy_2d_histo_1e-10s_steps"
#pio.write_html(fig, fname+".html")
pio.write_image(fig, fname+".pdf")
pio.write_image(fig, fname+".png")

In [ ]:
for coord in ["X","Y","Z","R"]:
    fig = histo(df[largeR_cut][coord],xlabel=f"{coord} (mm)",bins=50,autobin=False,show_plot=False,inline=True);
    fig.layout.title = ("Signal e- Trajectory, 1e-10 Time Steps <br> R_max > 400 mm, Z_max > 8410 mm")
    fig.update_xaxes(ticks="inside",nticks=13)
    fig.update_yaxes(ticks="inside")
    fname = plot_dir+f"run05_{coord}_histo_1e-10s_steps_largeR_largeZ"
    pio.write_image(fig,fname+".pdf")
    pio.write_image(fig,fname+".png")

### Kind of useless plots now that code is working :)

In [ ]:
grouped = df.groupby('particle')

In [ ]:
#zmax = grouped['Z'].agg(np.max)
zmax = grouped['Z'].last()

In [ ]:
histo(zmax,xlabel='Z_max (mm)',bins=15,inline=True);
histo(zmax[zmax<6290],xlabel='Z_max (mm)',bins=15,inline=True);
histo(zmax[zmax<8410],xlabel='Z_max (mm)',bins=15,inline=True);

In [ ]:
start_E = grouped['E0'].first()
end_E = grouped['EF'].last()

In [ ]:
e_change = end_E - start_E

In [ ]:
e_change.name = ""

In [ ]:
histo(e_change,xlabel=f'$\Delta E (MeV)$',yscale='linear',bins=30,inline=True);
histo(e_change,xlabel=f'$\Delta E (MeV)$',yscale='linear',bins=30,cut=[0.,.75],inline=True);

In [ ]:
data = [
    #go.Histogram2dContour(
    go.Scatter(
        x=zmax,
        y=e_change,
        mode='markers',
        #colorscale='Viridis',
        #bins
        #line=line(width=0.1)
    )
]

axis_template = dict(
    ticks='',
    showgrid=False,
    zeroline=False,
    showline=True,
    mirror=True,
    linewidth=2,
    linecolor='#444'
)

layout = go.Layout(xaxis=axis_template,
                   yaxis=axis_template,
                   width=800,
                   height=750,
                   autosize=False,
                   hovermode='closest',
                   title=''
)

fig = go.Figure(data=data, layout=layout)

In [ ]:
iplot(fig)

In [ ]:
start = grouped.first()
end = grouped.last()

In [ ]:
df_init

In [ ]:
start

In [ ]:
end

It seems interesting to pick out a certain particle that terminated early to look at a 3d plot of the trajectory. e.g. particle 1 ends in the graded region but only terminated because of hitting the maximum time.

In [ ]:
p1 = grouped.get_group(19)

In [ ]:
#p1 = p1.iloc[::10,:]

In [ ]:
df_init.query("particle == 19")

Very large initial Pt!

In [ ]:
p1

In [ ]:
fig = scatter3d(p1.X,p1.Y,p1.Z,scale_list=[p1.time],mode_list=['markers+lines'],units_list=[('mm','mm','mm')],inline=True);

In [ ]:
p1.Vx.plot(x='time')
p1.Vy.plot(x='time')
p1.Vz.plot(x='time')

Let's see how the energy changes in time.

In [ ]:
c = 299792458000 #mm/s
m = .511 #MeV/c^2

In [ ]:
p1.eval('v = (Vx**2+Vy**2+Vz**2)**(1/2)',inplace=True)
p1.eval(f'gamma = (1-(v/{c})**2)**(-1/2)',inplace=True)
p1.eval(f'p = gamma*{m}*v/{c}',inplace=True)

In [ ]:
p1

In [ ]:
#p1.eval(f"E = (p**2*{c}**2+{m}**2*{c}**4)**(1/2)",inplace=True)
p1.eval(f"E = (p**2+{m}**2)**(1/2)",inplace=True)

In [ ]:
p1.plot(x="time",y="E")

### More Useful Plots

In [ ]:
plt.hist(df.X,bins=50);
plt.title("Signal e- Trajectory, 1e-10 Time Steps")
plt.xlabel("X (mm)")
plt.ylabel('Count')
#plt.savefig(plot_dir+"run05_x_histo_1e-10s_steps.pdf")

In [ ]:
plt.hist(df.Y,bins=50);
plt.title("Signal e- Trajectory, 1e-10 Time Steps")
plt.xlabel("Y (mm)")
plt.ylabel('Count')
#plt.savefig(plot_dir+"run05_y_histo_1e-10s_steps.pdf")

In [ ]:
plt.hist(df.Z,bins=50);
plt.title("Signal e- Trajectory, 1e-10 Time Steps")
plt.xlabel("Z (mm)")
plt.ylabel('Count')
#plt.savefig(plot_dir+"run05_z_histo_1e-10s_steps.pdf")

In [ ]:
plt.hist(df.R,bins=50);
plt.title("Signal e- Trajectory, 1e-10 Time Steps")
plt.xlabel("Radius (mm)")
plt.ylabel('Count')
#plt.savefig(plot_dir+"run05_r_histo_1e-10s_steps.pdf")

In [ ]:
#fig = histo(df["X"],xlabel=f"X (mm)",bins=50,autobin=False,show_plot=True,inline=True);

In [ ]:
for coord in ["X","Y","Z","R"]:
    fig = histo(df[coord],xlabel=f"{coord} (mm)",bins=50,autobin=False,show_plot=False,inline=True);
    fig.layout.title = ("Signal e- Trajectory, 1e-10 Time Steps")
    fig.update_xaxes(ticks="inside",nticks=13)
    fig.update_yaxes(ticks="inside")
    fname = plot_dir+f"run05_{coord}_histo_1e-10s_steps"
    pio.write_image(fig,fname+".pdf")

### Etc

In [ ]:
df_mustops = pd.read_pickle(mu2e_ext_path+"root/ce_OLD.pkl")

In [ ]:
df_mustops.demcgen_x

In [ ]:
plt.hist2d(x=df_mustops.demcgen_x,y=df_mustops.demcgen_y,bins=50);

In [ ]:
#df_mustops.eval("demcgen_r = (demcgen_x**2 + demcgen_y**2)**(1/2)",inplace=True)
df_mustops["demcgen_r"] = (df_mustops.demcgen_x**2 + df_mustops.demcgen_y**2)**(1/2)#,inplace=True)
df_mustops = df_mustops[df_mustops['de._status'] != -1000]

In [ ]:
plt.hist(x=df_mustops.demcgen_r,bins=50);

In [ ]:
plt.hist(x=df_mustops.demcgen_y,bins=75);

In [ ]:
plt.hist(x=df_mustops.demcgen_x,bins=75);

## Explore pT

In [ ]:
largeR_tracks

In [ ]:
df_init2 = df_init[df_init.particle.isin(largeR_tracks)]

In [ ]:
#df_init2 = df_init[df_init["Theta0"] < np.pi/2]
#df_init2 = df_init2[df_init2["Theta0"] < np.pi/2]

In [ ]:
tand = abs(df_init2["Pz0"]/df_init2["Pt0"])
#tand = df_init2["Pz0"]/df_init2["Pt0"]

In [ ]:
tand.name = "tan(dip)"

In [ ]:
fig = histo(tand, bins=25, show_plot=True,inline=True)

In [ ]:
fig = histo(tand, bins=25, show_plot=True,inline=True)

In [ ]:
fig=histo(df_init.Theta0,bins=20,inline=True)

In [ ]:
#theta = np.random.uniform(0.1,np.pi/2,10000)
costheta = np.random.uniform(0.1,1,10000)
theta = np.arccos(costheta)

In [ ]:
theta

In [ ]:
dip = pd.Series(1/np.tan(theta), name="tan(dip)")

In [ ]:
fig = histo(dip, bins=1000, inline=True)

In [ ]:
dip.describe()

In [ ]:
lar

In [ ]:
df.columns

In [ ]:
df.head()

## tan(dip)

In [ ]:
c = 2.9979e11

In [ ]:
#c

In [ ]:
zmax = df.Z < 11740.
df = df[zmax]

In [ ]:
df.eval("V = (Vx**2+Vy**2+Vz**2)**(1/2)", inplace=True)
df.eval("Vt = (Vx**2+Vy**2)**(1/2)", inplace=True)
df.eval(f"gamma = (1 - (V/{c})**2)**(1/2)", inplace=True)
df.eval("Pt = gamma*0.511*Vt",inplace=True)
df.eval("Pz = gamma*0.511*Vz",inplace=True)
df.eval("P = gamma*0.511*V",inplace=True)

In [ ]:
#df.P

In [ ]:
zcut = grouped.R.max()[grouped.Z.max()>8410.]
zcutrcut = zcut[zcut > 450.]
largeR_tracks = list(zcutrcut.index.values)

In [ ]:
largeR_cut = df.particle.isin(largeR_tracks)

In [ ]:
grouped_largeR = df[largeR_cut].groupby("particle")

In [ ]:
last_largeR = grouped_largeR.tail(1)

In [ ]:
#last_largeR

In [ ]:
last_largeR.eval("tand = Pz / Pt",inplace=True)

In [ ]:
fig = histo(last_largeR.tand, bins=40, inline=True)

In [ ]:
fig.update_xaxes(range=[0.4,1.5])

In [ ]:
12/31847

In [ ]:
10/1264